In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import all the libraries needed

In [ ]:
print ('Load imports Starting')

import numpy as np
import pandas
import matplotlib.pyplot as plt
from datetime import datetime
from random import sample
from collections import defaultdict

from sklearn import tree, datasets, neighbors, svm, preprocessing
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, learning_curve
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler  

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', 'Solver terminated early.*')
warnings.filterwarnings("ignore")

print('Load imports Completed')

# Load the input data

In [ ]:
print('Load files Started')

# Local
# df_asteroid = pandas.read_csv('Datasets/Asteroids/dataset.csv')

# Kaggle
df_asteroid = pandas.read_csv('../input/asteroid-dataset/dataset.csv')

print('Load files Completed')

# Pre-process the data

In [ ]:
print ('Formatting Starting')
np.random.seed(0) 

df = None
df_x = None
df_y = None
to_encode = []

classifed_names = None

inputs = df_asteroid

inputs.pha.replace(('Y', 'N'), (1, 0), inplace = True)
inputs['pha'] = inputs['pha'].fillna(0)
inputs['pha'] = inputs.pha.astype(int)

inputs.neo.replace(('Y', 'N'), (1, 0), inplace = True)
inputs['neo'] = inputs['neo'].fillna(0)
inputs['neo'] = inputs.neo.astype(int)

inputs = inputs.drop(['id', 'spkid', 'full_name', 'name', 'prefix', 'orbit_id', 'pdes', 'equinox', 'diameter', 'albedo', 
                      'diameter_sigma'], axis='columns')
inputs.dropna(inplace=True)
target = inputs['class']

df = inputs
inputs1 = inputs.drop(['class'], axis='columns')
df_x = inputs1.columns
df_y = 'class'
to_encode = inputs1.columns
    
print(df)
le = preprocessing.LabelEncoder
encoderDict = defaultdict(le)
for column in to_encode:
    print('Encoding: ' + column)
    df[column] = df[column].dropna()
    df = df[df[column].notnull()]
    df[column] = encoderDict[column].fit_transform(df[column])
    print(encoderDict[column].classes_)

print(df_x)
print(df_y)

print(df.columns)
print(df.dtypes)
print ('Formatting Completed')

# Shrink the dataset to a manageable size - Top 100,000

In [ ]:
# Shrink the dataset to a manageable size
df = df.head(100000)
print('File sample reduced')

# Train Test split as 70/30

In [ ]:
print('Starting Train Test Split')
df = df.dropna()  

df = df.sample(frac=1).reset_index(drop=True)
print('Dataset size: ' + str(df.size))
print('Features: ' + str(df_x))
print('Target Decision: ' + df_y)

X = df.loc[:, df_x]
y = df.loc[:, df_y]

trainTest = train_test_split(X, y, test_size=0.3, train_size=0.7, random_state=0)
X_train, X_test, y_train, y_test = trainTest.copy()

name = 'Mortgage'
print('Train Test Split completed')

# Cross validation and plot used by all algorithms

In [ ]:
print ('Define Methods Starting')
cv = 4
trainSizes = [1000, 2500, 5000, 7500, 10000, 12500, 15000]

def plotLearningCurve(title, cv_curve):
    trainSizes, trainScores, testScores = cv_curve
    trainMean = np.mean(trainScores, axis=1) 
    testMean = np.mean(testScores, axis=1)
    
    plt.figure()
    plt.title(title)
    plt.ylim((.4, 1.01))
    plt.xlabel("Train Size")
    plt.ylabel("Score")
    plt.grid()
    plt.plot(trainSizes, trainMean, 'o-', color="brown", label="Train score")
    plt.plot(trainSizes, testMean, 'o-', color="b", label="Test Score CV=4")
    plt.legend(loc="best")
    return plt

def cvTest(name, clf, params, trainTest, scaler=None, plot=True):
    X_train, X_test, y_train, y_test = trainTest.copy()
    print('Name: ' + name)
    if not scaler is None:
        scaler.fit(X_train) 
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
    
    cv_clf = GridSearchCV(clf, param_grid=params, refit=True, cv=cv)
    
    start = datetime.now()
    cv_clf.fit(X_train, y_train)
    end = datetime.now()
    train_time = (end - start).total_seconds()
    train_score = cv_clf.score(X_train, y_train)
    
    start = datetime.now()
    test_score = cv_clf.score(X_test, y_test)
    end = datetime.now()
    test_time = (end - start).total_seconds()
    
    y_predict = cv_clf.predict(X_test)
    
    cvEstimator = cv_clf.best_estimator_
    
    result = '{:.4f}\t{:.4f}\t{:.4f}\t{:.4f}'.format(train_score, train_time, test_score, test_time)
    
    if plot:
        cv_curve = learning_curve(cvEstimator, X_train, y_train, cv=cv, train_sizes=trainSizes)
        plot = plotLearningCurve(name, cv_curve) 

    return (cvEstimator, cv_clf, result)

print("Define Methods complete")

# Decision Tree without Pruning
* Criterion: gini, entropy
* Max Depth: 1, 6, 10, 15, 25, 50

In [ ]:
print('Decision Tree without ccp alpha Starting')

finalResult = []
finalResult.append('Max Depth' + '\t' + 'Criterion' + '\t' + 'Tree Size' + '\t' +  'CCP alpha' + '\t' + 'Train Score' + '\t' + 'Train Time' + '\t' + 'Test Score' + '\t' + 'Test Time')

params = { 'criterion':['gini','entropy'] }
for i in [1, 6, 10, 15, 25, 50]:
       clf = tree.DecisionTreeClassifier(max_depth = i, class_weight='balanced', splitter='best', min_samples_leaf=1, random_state=0)
       returnValue, return_clf, result = cvTest('Decision Tree: ' + str(i), clf, params, trainTest)
       finalResult.append('{}\t{:10s}\t {}\t'.format(i, return_clf.best_params_['criterion'], returnValue.tree_.node_count) + result)

print('\n'.join(finalResult))
print ('Decision Tree without ccp alpha Completed')

# Decision Tree Pruning
* Criterion: gini, entropy
* Max Depth: 1, 6, 10, 15, 25, 50
* ccp_alpha: 0.0001, 0.001, 0.01, 0.1

In [ ]:
print('Decision Tree with ccp alpha Starting')

finalResult = []
finalResult.append('Max Depth' + '\t' + 'Criterion' + '\t' + 'Tree Size' + '\t' +  'CCP alpha' + '\t' + 'Train Score' + '\t' + 'Train Time' + '\t' + 'Test Score' + '\t' + 'Test Time')

params = { 'criterion':['gini','entropy'] }
for i in [1, 6, 10, 15, 25, 50]:
       for ccp_alpha in [0.0001, 0.001, 0.01, 0.1]:
          clf = tree.DecisionTreeClassifier(max_depth = i, class_weight='balanced', splitter='best', min_samples_leaf=1, random_state=0,ccp_alpha=ccp_alpha)
          returnValue, return_clf, result = cvTest('Decision Tree: ' + str(i) + ' ' + str(ccp_alpha), clf, params, trainTest)
          finalResult.append('{}\t{:10s}\t{}\t{}\t'.format(i, return_clf.best_params_['criterion'], returnValue.tree_.node_count, ccp_alpha) + result)

print('\n'.join(finalResult))
print ('Decision Tree with ccp alpha Completed')

# Artificial Neural Network - MLP Classifier
* Solver: adam, sgd
* Alpha: .0001, .005, .001
* Learning rate init: .001, .01, .1, .2

In [ ]:
print('ANN Starting')

finalResult = []
finalResult.append('Solver' + '\t' + 'Iter' + '\t' + ' Alpha' + '\t' + 'Learning Rate' + '\t' + 'Train Score' + '\t' + 'Train Time' + '\t' + 'Test Score' + '\t' + 'Test Time')

alphas = [0.0001, 0.005, 0.001]
learning_rate_inits = [ 0.001, .01, .1, .2]
        
params = {}

for solver in ['adam', 'sgd']:
    for alpha in alphas:
        for learning_rate_init in learning_rate_inits:
            clf = MLPClassifier(solver=solver, max_iter=20, random_state=0, batch_size='auto')
            plot = True
            returnValue, return_clf, result = cvTest(solver + ' ' + str(alpha) + '/' + str(learning_rate_init), clf, params, trainTest, StandardScaler(), plot)
            finalResult.append('{}\t{}\t{:.4f}\t{:.4f}\t'.format(solver, str(returnValue.n_iter_), alpha, learning_rate_init) + result) 

print('\n'.join(finalResult))
print ('ANN complete')

# Neural Network - MLP Classifier - Loss Curve Plot
* Solver: sgd, adam
* Learning Rate: constant, invscaling
* Momentum: 0, .9
* Learning rate init: .01, .2

In [ ]:
print('ANN Plot Starting')

finalResult = []
finalResult.append('{:20s}'.format('Label') + '\t' + '\t' + '\t' + '\t' + 'Score' + '\t' + 'Loss' + '\t' + 'Train Time')

params = [{'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9,
           'nesterovs_momentum': False, 'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'constant', 'momentum': .9,
           'nesterovs_momentum': True, 'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': .9,
           'nesterovs_momentum': True, 'learning_rate_init': 0.2},
          {'solver': 'sgd', 'learning_rate': 'invscaling', 'momentum': .9,
           'nesterovs_momentum': False, 'learning_rate_init': 0.2},
          {'solver': 'adam', 'learning_rate_init': 0.01}]

labels = ["constant with momentum", "constant with Nesterov's momentum",
          "inv-scaling with momentum", "inv-scaling with Nesterov's momentum", "adam"]

plot_args = [{'c': 'red', 'linestyle': '-'},
             {'c': 'green', 'linestyle': '-'},
             {'c': 'orange', 'linestyle': '--'},
             {'c': 'purple', 'linestyle': '--'},
             {'c': 'black', 'linestyle': '-'}]

def plotDataset(X, y, ax, name):
    ax.set_title(name)
    X = MinMaxScaler().fit_transform(X)
    mlps = []
    max_iter = 20

    for label, param in zip(labels, params):
        mlp = MLPClassifier(verbose=0, random_state=0,
                            max_iter=max_iter, **param)
        
        start = datetime.now()
        mlp.fit(X, y)
        end = datetime.now()
        train_time = (end - start).total_seconds()
        
        start = datetime.now()
        score = mlp.score(X, y)
        end = datetime.now()
        score_time = (end - start).total_seconds()
        
        finalResult.append('{:40s}\t{:.4f}\t{:.4f}\t{:.4f}'.format(label, score, mlp.loss_, train_time))
        
        mlps.append(mlp)

    for mlp, label, args in zip(mlps, labels, plot_args):
        ax.plot(mlp.loss_curve_, label=label, **args)

fig, axes = plt.subplots(1, 1, figsize=(15, 5))

ax = axes
plotDataset(X, y, ax, 'Loss Curve')

plt.xlabel("Iterations")
plt.ylabel("Loss")
fig.legend(ax.get_lines(), labels, ncol=1, loc="upper right")
plt.show()

print('\n'.join(finalResult))
print ('ANN Plot Complete')

# K-Nearest Neighbor
* Weight: uniform, distance
* k-value: 1, 5, 10, 15, 20, 50, 100

In [ ]:
finalResult = []
finalResult.append('Weight' + '\t' + 'k-value' + '\t' + 'Train Score' + '\t' + 'Train Time' + '\t' + 'Test Score' + '\t' + 'Test Time')

print('KNN Starting')
weights = ['uniform', 'distance']
k_vals = [1, 5, 10, 15, 20, 50, 100]

for weight in weights:
    for k in k_vals:
        clf = neighbors.KNeighborsClassifier(k, weights=weight)
        returnValue, return_clf, result = cvTest('KNN - ' + weight + ': ' + str(k), clf, {}, trainTest)
        finalResult.append('{}\t{}\t '.format(weight, k) + result)

print('\n'.join(finalResult))
print ('KNN Completed')

# Support Vector Machine
* Kernel: rbf, poly
* Gamma: .01, .1, .5, .7, .9. 1

In [ ]:
print('SVM Starting')

finalResult = []
finalResult.append('Kernel' + '\t' + 'Gamma' + '\t' + 'Train Score' + '\t' + 'Train Time' + '\t' + 'Test Score' + '\t' + 'Test Time')

kernels = ['rbf', 'poly']
gammas = [0.01, .10, .50, .70, .90, 1]

for kernel in kernels:
    for gamma in gammas:
        clf_1 = svm.SVC(kernel=kernel, max_iter=3000, gamma=gamma)
        returnValue, return_clf, result = cvTest('SVM ' + kernel + ' - ' + str(gamma), clf_1, {}, trainTest, StandardScaler())
        finalResult.append('{}\t{:.2f}\t'.format(kernel, gamma) + result)

print('\n'.join(finalResult))
print ('SVM Completed')

# Adaboost
* Estimators: 1, 50, 100, 150
* Learning Rate: .1, 1, 10
* Max Depth: 1, 3, 10, 15, 25, 50

In [ ]:
print('Adaboost Starting')

finalResult = []
finalResult.append('Max Depth' + '\t' + 'Learning Rate' + '\t' + ' Esimtators' + '\t' + 'Train Score' + '\t' + 'Train Time' + '\t' + 'Test Score' + '\t' + 'Test Time')

estimators = [1, 50, 100, 150]
learning_rate = [.1, 1, 10]

for i in [1, 3, 10, 15, 25, 50]:
    clf_base = tree.DecisionTreeClassifier(max_depth = i, class_weight='balanced', splitter='best', min_samples_leaf=1)
    clf = AdaBoostClassifier(base_estimator=clf_base)
    returnValue, return_clf, result = cvTest('Adaboost: ' + str(i), clf, {'n_estimators': estimators, 'learning_rate': learning_rate}, trainTest)
    finalResult.append('{}\t{:.1f}\t{}\t'.format(i, return_clf.best_params_['learning_rate'], return_clf.best_params_['n_estimators']) + result)

print('\n'.join(finalResult))
print('Adaboost Completed')